<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Armazenamento e Gestão de Dados com Data Lake e Data Lakehouse</font>
## <font color='blue'>Lab 6</font>
### <font color='blue'>Operações CRUD e Time Travel com Delta Lake</font>
### <font color='blue'>Exemplo 1</font>

In [1]:
# Imports
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

In [2]:
# Configuração do Spark com Delta Lake
builder = SparkSession.builder \
    .appName("Lab6Exemplo1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", "5000")  

In [4]:
# Cria a sessão
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [5]:
# Caminho da tabela Delta
delta_table_path = "/repositorio/dsa-delta-table-exemplo1"

## Operação CREATE

Operação de Insert também se enquadra na categoria de CREATE.

In [6]:
# Cria o dataframe
dsa_dados = spark.createDataFrame([(1, "Maria", 30),
                                   (2, "Bob", 25),
                                   (3, "Zico", 35)], 
                                  ["id", "nome", "idade"])

In [7]:
type(dsa_dados)

pyspark.sql.dataframe.DataFrame

In [8]:
# Cria tabela Delta
dsa_dados.write.format("delta").mode("overwrite").save(delta_table_path)

## Operação READ

In [9]:
# Leitura dos dados a partir da tabela
spark.read.format("delta").load(delta_table_path).show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   30|
|  3| Zico|   35|
|  2|  Bob|   25|
+---+-----+-----+



In [10]:
# Leitura simples dos dados no formato Delta
dados = spark.read.format("delta").load(delta_table_path)
dados.show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   30|
|  3| Zico|   35|
|  2|  Bob|   25|
+---+-----+-----+



In [11]:
type(dados)

pyspark.sql.dataframe.DataFrame

In [12]:
# Leitura dos dados filtrando por idade maior que 30
dados_filtrados = spark.read.format("delta").load(delta_table_path).filter("idade > 30")
dados_filtrados.show()

+---+----+-----+
| id|nome|idade|
+---+----+-----+
|  3|Zico|   35|
+---+----+-----+



In [13]:
# Registrar a tabela Delta como uma tabela temporária
dados = spark.read.format("delta").load(delta_table_path)
dados.createOrReplaceTempView("tabela_temp")

# Consulta SQL
spark.sql("SELECT * FROM tabela_temp WHERE nome = 'Maria'").show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   30|
+---+-----+-----+



In [14]:
# Definir o esquema esperado para validação ou transformação
esquema = StructType([
    StructField("id_cliente", IntegerType(), True),
    StructField("nome_cliente", StringType(), True),
    StructField("idade_cliente", IntegerType(), True)
])

# Ler os dados e ajustar o esquema, se necessário
dados_ajustados = spark.read.format("delta").load(delta_table_path)
dados_ajustados = spark.createDataFrame(dados_ajustados.rdd, esquema)
dados_ajustados.show()

+----------+------------+-------------+
|id_cliente|nome_cliente|idade_cliente|
+----------+------------+-------------+
|         1|       Maria|           30|
|         3|        Zico|           35|
|         2|         Bob|           25|
+----------+------------+-------------+



In [15]:
# Adicionar uma nova coluna derivada durante a leitura
dados_modificados = spark.read.format("delta").load(delta_table_path).withColumn("idade_mais_5", col("idade") + 5)
dados_modificados.show()

+---+-----+-----+------------+
| id| nome|idade|idade_mais_5|
+---+-----+-----+------------+
|  1|Maria|   30|          35|
|  3| Zico|   35|          40|
|  2|  Bob|   25|          30|
+---+-----+-----+------------+



## Operação UPDATE

In [16]:
# Carrega a tabela Delta
delta_table = DeltaTable.forPath(spark, delta_table_path)

In [17]:
type(delta_table)

delta.tables.DeltaTable

In [18]:
# Isso aqui gera erro
#delta_table.show()

In [19]:
# Converte em dataframe e então mostra os dados
delta_table.toDF().show()

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   30|
|  3| Zico|   35|
|  2|  Bob|   25|
+---+-----+-----+



In [20]:
# Atualiza idade de Bob
delta_table.update(condition = "nome = 'Bob'",
                   set = {"idade": "30"})

In [21]:
print("Após atualização:")
delta_table.toDF().show()

Após atualização:


[Stage 54:=================================================>      (44 + 6) / 50]

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   30|
|  3| Zico|   35|
|  2|  Bob|   30|
+---+-----+-----+



In [22]:
# Atualiza a idade de todas as pessoas com idade menor que 35
delta_table.update(
    condition = "idade < 35",
    set = {"idade": "idade + 5"}  # Incrementa a idade em 5
)

print("Após atualização com idade < 35:")
delta_table.toDF().show()

Após atualização com idade < 35:


[Stage 84:=================================================>      (44 + 6) / 50]

+---+-----+-----+
| id| nome|idade|
+---+-----+-----+
|  1|Maria|   35|
|  3| Zico|   35|
|  2|  Bob|   35|
+---+-----+-----+



In [23]:
# Atualiza múltiplos campos com base em uma condição
delta_table.update(
    condition = "id = 1",
    set = {
        "idade": "40",
        "nome": "'Maria Atualizada'"
    }
)

print("Após atualização de múltiplos campos:")
delta_table.toDF().show()

Após atualização de múltiplos campos:


[Stage 101:=============================================>         (41 + 8) / 50]

+---+----------------+-----+
| id|            nome|idade|
+---+----------------+-----+
|  1|Maria Atualizada|   40|
|  3|            Zico|   35|
|  2|             Bob|   35|
+---+----------------+-----+



In [24]:
# Define a idade como 42 para aqueles cujo nome começa com a letra 'Z'
delta_table.update(
    condition = "nome LIKE 'Z%'",
    set = {"idade": "42"}
)

print("Após atualização para nomes começando com 'Z':")
delta_table.toDF().show()

Após atualização para nomes começando com 'Z':


+---+----------------+-----+
| id|            nome|idade|
+---+----------------+-----+
|  1|Maria Atualizada|   40|
|  3|            Zico|   42|
|  2|             Bob|   35|
+---+----------------+-----+



In [25]:
# Adiciona um sufixo ao nome das pessoas com idade maior ou igual a 42
delta_table.update(
    condition = "idade >= 42",
    set = {"nome": "CONCAT(nome, '_Senior')"}
)

print("Após atualização com sufixo no nome:")
delta_table.toDF().show()

Após atualização com sufixo no nome:


+---+----------------+-----+
| id|            nome|idade|
+---+----------------+-----+
|  1|Maria Atualizada|   40|
|  3|     Zico_Senior|   42|
|  2|             Bob|   35|
+---+----------------+-----+



In [26]:
# Define a idade como 50 para todas as linhas
delta_table.update(
    condition = "true",  # Sem condição, aplica a todos
    set = {"idade": "50"}
)

print("Após atualização sem condição:")
delta_table.toDF().show()

Após atualização sem condição:


[Stage 149:==================================================>    (46 + 4) / 50]

+---+----------------+-----+
| id|            nome|idade|
+---+----------------+-----+
|  1|Maria Atualizada|   50|
|  3|     Zico_Senior|   50|
|  2|             Bob|   50|
+---+----------------+-----+



## Operação DELETE

In [27]:
# Remove registros com id igual a 1
delta_table.delete(condition = "id = 1")

In [28]:
print("Após exclusão:")
delta_table.toDF().show()

Após exclusão:


+---+-----------+-----+
| id|       nome|idade|
+---+-----------+-----+
|  3|Zico_Senior|   50|
|  2|        Bob|   50|
+---+-----------+-----+



## Adicionando Colunas na Tabela Delta

É possível adicionar coluna a uma tabela Delta?

No Delta Lake, não há suporte direto (ainda) para adicionar uma nova coluna diretamente via a API de manipulação de tabelas (DeltaTable). Porém, você pode adicionar uma nova coluna ao DataFrame em memória e sobrescrever a tabela Delta com o esquema atualizado.

A solução mais comum é habilitar a opção mergeSchema ao sobrescrever os dados na tabela Delta. Isso permite a adição de novas colunas ao esquema existente.

In [29]:
from pyspark.sql.functions import lit

# Criar um novo DataFrame com uma nova coluna a partir do dataframe inicial
df_com_nova_coluna = dados.withColumn("nova_coluna", lit("default"))

# Usar mergeSchema para atualizar a tabela com a nova coluna
df_com_nova_coluna.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .save(delta_table_path)

In [30]:
print("Após adicionar nova coluna com mergeSchema:")
spark.read.format("delta").load(delta_table_path).show()

Após adicionar nova coluna com mergeSchema:


+---+-----------+-----+-----------+
| id|       nome|idade|nova_coluna|
+---+-----------+-----+-----------+
|  3|Zico_Senior|   50|    default|
|  2|        Bob|   50|    default|
+---+-----------+-----+-----------+



Se você preferir não usar a opção mergeSchema todas as vezes, pode ativá-lo como configuração de sessão:

In [31]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

In [32]:
# Grava o novo dataframe com a nova coluna
df_com_nova_coluna.write.format("delta") \
    .mode("overwrite") \
    .save(delta_table_path)

In [33]:
print("Após adicionar nova coluna com configuração global:")
spark.read.format("delta").load(delta_table_path).show()

Após adicionar nova coluna com configuração global:


+---+-----------+-----+-----------+
| id|       nome|idade|nova_coluna|
+---+-----------+-----+-----------+
|  3|Zico_Senior|   50|    default|
|  2|        Bob|   50|    default|
+---+-----------+-----+-----------+



## Inserindo Novos Dados na Tabela Delta

In [34]:
# Novo registro a ser adicionado
novo_registro = [(4, "Ana", 28)]

In [35]:
# Cria um DataFrame para o novo registro
novo_df = spark.createDataFrame(novo_registro, ["id", "nome", "idade"])

In [36]:
# Adiciona o novo registro à tabela Delta
novo_df.write.format("delta").mode("append").save(delta_table_path)

In [37]:
print("Após inserir novo registro:")
spark.read.format("delta").load(delta_table_path).show()

Após inserir novo registro:


+---+-----------+-----+-----------+
| id|       nome|idade|nova_coluna|
+---+-----------+-----+-----------+
|  3|Zico_Senior|   50|    default|
|  2|        Bob|   50|    default|
|  4|        Ana|   28|       NULL|
+---+-----------+-----+-----------+



# Fim